In [1]:
#5-2
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

C:\Users\ryosu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#5-3
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# 5-4
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [4]:
# 5-5
# 文章を符号化し、GPUに配置する
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

# BERTに入力し、分類スコアを得る。
# 系列長をそろえる必要がないので、単にinput_idsのみ入力する。
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

In [6]:
print(scores)

tensor([[[ -5.8543,   5.0460,  -1.7982,  ...,  -4.8385,  -6.4237,  -7.8076],
         [ -4.0240,   7.2824,  -5.3991,  ...,  -6.0372,  -6.5835,  -2.1293],
         [ -5.8373,   5.3621,  -2.2092,  ...,  -4.3541,  -5.7282,  -4.3881],
         ...,
         [ -7.8706,   5.9728,  -4.3934,  ...,  -4.3237,  -6.0895, -11.4354],
         [ -5.4528,   6.5476,   0.0342,  ...,  -4.5627,  -5.1644,  -7.0159],
         [ -8.7527,   3.2642,  -1.6652,  ...,  -5.0585,  -7.0542, -10.7617]]],
       device='cuda:0')


In [5]:
# 5-6
# ID列で'[MASK]'(IDは4)の位置を調べる。
mask_position = input_ids[0].tolist().index(4)

# スコアが最も良いトークンのIDを取り出し、トークンに変換する。
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

# [MASK]を上で求めたトークンで置き換える。
text = text.replace('[MASK]', token_best)

print(text)

今日は東京へ行く。


In [7]:
# 5-7
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    '''
    文章中の最初の[MASK]を上位のトークンに置き換える
    上位何位まで使うかは、num_topkで指定
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト
    '''
    # 文章を符号化し、BERTで分類スコアを得る
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits
    
    # スコアが上位のトークンとスコアを求める
    mask_position = input_ids[0].tolist().index(4)
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices  # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)  # トークン
    scores_topk = topk.values.cpu().numpy()  # スコア
    
    # 文章中の[MASK]を上で求めたトークンに置き換える
    text_topk = []
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))
    
    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。
